In [42]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (125 kB/s)
Reading package lists... Done


In [43]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Home_Entertainment").getOrCreate()

In [86]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [44]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [45]:
number_of_records = df.count()
number_of_columns = len(df.columns)
print(f'Home Entertainment Review Dataset \nhas\n{number_of_records} records\nand \n{number_of_columns} columns')

Home Entertainment Review Dataset 
has
705889 records
and 
15 columns


In [ ]:
#import module that deal with data type
from pyspark.sql.types import StructType, StructField,IntegerType,StringType,DateType

In [126]:
review_id_table=df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431| 2015-08-31|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622| 2015-08-31|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132| 2015-08-31|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707| 2015-08-31|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [127]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [142]:
schema = StructType([
  StructField("review_id", StringType(), False),
  StructField("customer_id", IntegerType(), True),
  StructField("product_id", StringType(), True),
  StructField("product_parent", IntegerType(), True),
  StructField("review_date", DateType(), True)])

review_id_table=spark.createDataFrame(data=review_id_table.rdd,schema=schema)
review_id_table.printSchema()

root
 |-- review_id: string (nullable = false)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [143]:
products=df.select(['product_id','product_title'])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00NTI0CQ2|Aketek 1080P LED ...|
|B00BUCLVZU|TiVo Mini with IR...|
|B00RBX9D5W|Apple TV MD199LL/...|
|B00UJ3IULO|New Roku 3 6.5 Fo...|
|B0037UCTXG|Generic DVI-I Dua...|
+----------+--------------------+
only showing top 5 rows



In [144]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [145]:
schema = StructType([
  StructField("product_id", StringType(), False),
  StructField("product_title", IntegerType(), True)])


products=spark.createDataFrame(data=products.rdd,schema=schema)
products.printSchema()

root
 |-- product_id: string (nullable = false)
 |-- product_title: integer (nullable = true)



In [53]:
customers = df.select(['customer_id']).groupBy('customer_id').count()
customers = customers.withColumnRenamed('count', 'customer_count')
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   31325270|             1|
|   52857540|             1|
|   46767124|             1|
|   13595560|             1|
|   23478301|             2|
+-----------+--------------+
only showing top 5 rows



In [146]:
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)



In [147]:
schema = StructType([
  StructField("customer_id", StringType(), False),
  StructField("customer_count", IntegerType(), True)])


customers=spark.createDataFrame(data=customers.rdd,schema=schema)
customers.printSchema()

root
 |-- customer_id: string (nullable = false)
 |-- customer_count: integer (nullable = true)



In [54]:
vine_table = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RY01SAV7HZ8QO|          4|            0|          0|   N|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|
|R149Q3B5L33NN5|          5|            0|          0|   N|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [148]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [149]:
schema = StructType([
  StructField("review_id", StringType(), False),
  StructField("start_rating", IntegerType(), True),
  StructField("helful_votes", IntegerType(), True),
  StructField("total_votes", IntegerType(), True),
  StructField("vine", StringType(), True)])


customers=spark.createDataFrame(data=customers.rdd,schema=schema)
customers.printSchema()

root
 |-- review_id: string (nullable = false)
 |-- start_rating: integer (nullable = true)
 |-- helful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [150]:
#create table schemas on AWS RDS via PgAdmins
#deal with the data on AWS S3 buckets to create data for each tables
#input data into AWS RDS by using jdbc

In [166]:
# Configure settings for RDS
mode = "append"
# jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
# connection string>>>AWS RDS endpoints
# database-name>>>pgadmin DB name
jdbc_url="jdbc:postgresql://amazon-review-instance.cj2lqy9pzbrt.us-east-2.rds.amazonaws.com:5432/amazon_review_db"

config = {"user":"root", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [167]:
#Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [155]:
#Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [156]:
#Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [157]:
#Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored